# Automatic import of Basel drinking fountains data to Wikidata
The following script downloads fountain data from Open Data swiss as per https://github.com/water-fountains/import2wikidata/issues/12, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


Python v 3.6.5


In [2]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   "http://query.wikidata.org/sparql"
   ,prefixes=prefixes
)

## Load data

In [3]:
df = pd.read_csv("data/Trinkbrunnen.csv")

In [4]:
df.head()

,X,Y,Name,Description
0,7.629450,47.562891,Äänisbreetli-Brunnen,"<img src=""https://doc-04-24-mymaps.googleuserc..."
1,7.597086,47.558396,Achteck-Brunnen,"<img src=""https://doc-0s-24-mymaps.googleuserc..."
2,7.593189,47.550235,Aeschengraben-Brunnen,"<img src=""https://doc-00-24-mymaps.googleuserc..."
3,7.586400,47.558155,Affen-Brunnen,"<img src=""https://doc-0g-24-mymaps.googleuserc..."
4,7.593095,47.554276,Affen-Brunnen,"<img src=""https://doc-0g-24-mymaps.googleuserc..."


### Rename columns to make them easier to work with

In [5]:

# remove not needed columns
df = df.drop(columns=['Description'])
# rename columns
df = df.rename(index=str, columns=
               {"Name": "label_de"})

## Identify already existing fountains
### Query fountains from Wikidata

In [6]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}

In [26]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above

query_string = """ SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
}}
  """.format(**bounds)

# Perform query
query_result = endpoint.select(query_string)

In [27]:
print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(query_result.size))

 SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{
  # Enter coordinates
  SERVICE wikibase:box {
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point(7.5570532 47.5279767)"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point(7.630431 47.5840738)"^^geo:wktLiteral.
  } .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  } 
  OPTIONAL { ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}
  OPTIONAL { ?place wdt:P571 ?date.}
  OPTIONAL { ?place wdt:P5623 ?water_supply_type}
  OPTIONAL { ?place wdt:P137 ?operator.}
}
  


Total number of rows incl. duplicates 2528


### Tidy up data

In [28]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q61881745,Allschwilerplatz-Brunnen,Point(7.5693423 47.5574918),1933-01-01T00:00:00Z,None,None,None,None,7.569342,47.557492
1,wd:Q72935373,Brunnen im Bruderholzschulhaus,Point(7.595505 47.5309589),None,None,None,wd:Q72936279,None,7.595505,47.530959
2,wd:Q72935434,Sevogel-Brunnen,Point(7.5887686 47.5587034),None,None,None,wd:Q72936279,None,7.588769,47.558703
3,wd:Q72935468,Stachelschützen-Brunnen,Point(7.5826585 47.558526),None,None,None,wd:Q72936279,None,7.582658,47.558526
4,wd:Q72935495,Brunnen (Seelöwe-Planschbecken ),Point(7.5740674 47.55292),None,None,None,wd:Q72936279,None,7.574067,47.552920
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q72954527,Stock-Brunnen,Point(7.6176131 47.5731528),None,None,None,wd:Q72936279,None,7.617613,47.573153
96,wd:Q72954550,Pausenhof-Brunnen,Point(7.5924915 47.565687),None,None,None,wd:Q72936279,None,7.592492,47.565687
97,wd:Q72954575,Pausenhof-Brunnen,Point(7.6038963 47.5477333),None,None,None,wd:Q72936279,None,7.603896,47.547733
98,wd:Q72954599,Pausenhof-Brunnen,Point(7.5707629 47.5626293),None,None,None,wd:Q72936279,None,7.570763,47.562629


In [29]:
# show duplicates
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated

ids = query_result['placeLabel']
duplicates = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
dupli = duplicates.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
dupli.sort_values(by=['placeLabel'],inplace=True)

print(dupli.to_string())

            place                      placeLabel         X          Y
0    wd:Q61881745        Allschwilerplatz-Brunnen  7.569342  47.557492
16   wd:Q72950812        Allschwilerplatz-Brunnen  7.568971  47.557485
254  wd:Q72952227                  Biotop-Brunnen  7.604020  47.547087
257  wd:Q72952269                  Biotop-Brunnen  7.583874  47.558209
41   wd:Q72951243              Brunnen (Basilisk)  7.588828  47.575791
42   wd:Q72951265              Brunnen (Basilisk)  7.593130  47.558946
43   wd:Q72951282              Brunnen (Basilisk)  7.588496  47.563151
44   wd:Q72951303              Brunnen (Basilisk)  7.586427  47.567261
46   wd:Q72951338              Brunnen (Basilisk)  7.574856  47.551436
48   wd:Q72951377              Brunnen (Basilisk)  7.592379  47.563197
40   wd:Q72951232              Brunnen (Basilisk)  7.591596  47.576851
49   wd:Q72951396              Brunnen (Basilisk)  7.602357  47.557118
50   wd:Q72951417              Brunnen (Basilisk)  7.583245  47.568577
51   w

In [30]:
def createMerge(dupli, extraText):
    #Create MERGE Command
    linesDup = []
    prevQ= ""
    prevX=0
    prevY=0
    for index, row in dupli.iterrows():
        
        # either create new or edit existing entity
        if row['X'] == prevX:
            if row['Y'] == prevY:
                lineDup = "MERGE\t"+prevQ[3:]+"\t"+row['place'][3:]+"\n"
                linesDup.append(lineDup)
            else:
                prevX=row['X']
                prevY=row['Y']
                prevQ=row['place']
        else:
            prevX=row['X']
            prevY=row['Y']
            prevQ=row['place']
            
    print("Merge commands"+extraText)
    print(linesDup)
    
createMerge(dupli, "")    

Merge commands
['MERGE\tQ72965554\tQ72967108\n', 'MERGE\tQ72965929\tQ72967566\n']


In [19]:
#write MERGE command to File
quickStatDupliFileName = "quickstatement_commands_BS_drink_DUPLI_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatDupliFileName, "w", encoding='utf8') as f:
    f.writelines(linesDup)
print("wrote '"+quickStatDupliFileName+"' with "+str(len(linesDup))+" lines")

wrote 'quickstatement_commands_BS_drink_DUPLI_191101_070225.txt' with 2 lines


In [25]:
# show duplicates STRICT
#idsS = query_result['placeLabel','X','Y'] #this does not work yet
#duplicatesS = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
#dupliS = duplicatesS.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
#dupliS.sort_values(by=['placeLabel'],inplace=True)
#print(dupliS.to_string())
#createMerge(dupliS, " - strict")  

KeyError: ('placeLabel', 'X', 'Y')

In [ ]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')
print("\n\nTotal number of rows without duplicates "+str(query_result.size))

### Compute distances between fountains

In [ ]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each OpenData.swiss basel fountain

In [ ]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

### Find out what information already exists for the nearest fountains

In [ ]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [ ]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [ ]:
dffinal.head()

In [ ]:
print("\n\nTotal number of rows "+str(dffinal.size))

## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [ ]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label_de(text):
    # process German language labels
    if text is None:
        return ''
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"Brunnen ({})"'.format(text)
    

def createline(lines, item, prop, value, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [ ]:
# initialize command storage list
lines = []

for index, row in dffinal.iterrows():
    
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of drinking fountain
        lines = createline(lines, item, 'P31', 'Q1630622')

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']))
        
        # TODO put reference "https://opendata.swiss/dataset/bade-trinkwasser-und-zierbrunnen-in-basel"
        
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    if item == 'LAST' or not row['nearest_has_label_de']:
        lines = createline(lines, item, 'Lde', process_label_de(row['label_de']))
    
    # creation date
    #if item == 'LAST' or not row['nearest_has_date']:
    #    lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by IWB  
    lines = createline(lines, item, 'P137', 'Q72936279')
            
    # catalog number can always be added (it is hard to check for)
    #lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
    #    'prop': 'P972',
    #    'value': 'Q53629101'
    #}])

# Write commands to file

In [ ]:
quickStatFileName = "quickstatement_commands_BS_drink_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.